In [14]:
import json
import GPy
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from os.path import join, splitext
from os import listdir
from multiprocessing import Pool,Lock
import threading

In [56]:
data_path = '/home/rodrigo/Documents/curvas-de-luz-EL4106/OGLE_trainingDB(1)/'
sub_class = 'CEPH/F_10'

In [16]:
# para tener todos los directorios, bastaria eliminar los que ya se procesaron.
join('asd','fg','ed')

'asd/fg/ed'

In [55]:
def test(lista):
    print( lista[2],'\n' )
l1 = [2,3,4]
l2 = [3,4,5]
l4 = [5,6]

t2 = threading.Thread(target=test, args=[l1])
t2.start()
t2.join()
    

t3 = threading.Thread(target=test, args=[l2])
t3.start()
t3.join()
try:
    t4 = threading.Thread(target=test, args=[l4[2]])
    t4.start()
    t4.join()
except Exception:
    _


4 
5
 



In [46]:
lock = Lock()
file = open('database2.csv','a')
def smooth(x, window_len=11, window='hanning'):
    if window_len < 3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y    

In [ ]:
def worker(data,period):
    mjd, mag_r, err_r, mag_b, err_b = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4]
    """
    mjd_b = mjd
    mjd_r = mjd
    """
    mjd_b = mjd[err_b < 9.999]
    mjd_r = mjd[err_r < 9.999]
    mag_b = mag_b[err_b < 9.999]
    mag_r = mag_r[err_r < 9.999]
    err_b = err_b[err_b < 9.999]
    err_r = err_r[err_r < 9.999]
    
    mask = err_b < np.mean(err_b) + 3*np.std(err_b)
    mjd_b = mjd_b[mask]
    mag_b = mag_b[mask]
    err_b = err_b[mask]
    mask = err_r < np.mean(err_r) + 3*np.std(err_r)
    mjd_r = mjd_r[mask]
    mag_r = mag_r[mask]
    err_r = err_r[mask]

    phi = np.mod(mjd_b, period)/period
    I = np.argsort(phi)
    phi_rs = np.linspace(0, 1, num=200)
    mag_b_rs = np.interp(phi_rs, phi[I], mag_b[I])


    kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m = GPy.models.GPHeteroscedasticRegression(phi[:, np.newaxis], mag_b[:, np.newaxis], kernel)
    m['.*het_Gauss.variance'] = err_b[:, np.newaxis]**2 
    m.het_Gauss.variance.fix() 
    #m.het_Gauss.period.fix() 
    
    m.optimize()
    m.optimize(messages=True)
    m.optimize_restarts(num_restarts = 5)
    mu, var = m._raw_predict(phi_rs[:, np.newaxis])
    # File writer
    mu = json.dumps(mu[:,0].tolist())
    file.write(mu + ';' + subclass+'\n')
    

In [57]:
dir_list = listdir(data_path)
for dir in dir_list:
    for subclass in listdir(join(data_path,dir)):
        lc_list = [splitext(file)[0] for file in sorted(listdir(join(data_path, dir , subclass))) if splitext(file)[1] == '.dat']
        print(dir,subclass,len(lc_list))
        if len(lc_list)>99999950:
            for lc_index in range(1,len(lc_list),4):
                data = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.time'))
                period = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.dat'))
                try:
                    data2 = np.loadtxt(join(data_path, sub_class, lc_list[lc_index+1]+'.time'))
                    period2 = np.loadtxt(join(data_path, sub_class, lc_list[lc_index+1]+'.dat'))
                    t2 = threading.Thread(target=work, args=[data2,period2,file])
                    t2.start()
                    t2.join()
                except Exception:
                    _
                
                try:
                    data3 = np.loadtxt(join(data_path, sub_class, lc_list[lc_index+2]+'.time'))
                    period3 = np.loadtxt(join(data_path, sub_class, lc_list[lc_index+2]+'.dat'))
                    t3 = threading.Thread(target=work, args=[data3,period3,file])
                    t3.start()
                    t3.join()
                except Exception:
                    _
                try:
                    data4 = np.loadtxt(join(data_path, sub_class, lc_list[lc_index+3]+'.time'))
                    period4 = np.loadtxt(join(data_path, sub_class, lc_list[lc_index+3]+'.dat'))
                    t4 = threading.Thread(target=work, args=[data4,period4,file])
                    t4.start()
                    t4.join()
                except Exception:
                    _
                work(data,period,file)
            
            
            
        
file.close()
        


    


RRL RRab 11701
EB ED_ESD 319
EB ELL_EC 147
EB EC 721
EB ED_VAR 58
EB ESD 3240
EB ED_TEB 3
EB ESD_DPV 12
EB ED 4755
CEPH 1O 1027
CEPH 1O_20 166
CEPH 1O_3O 2
CEPH F_10_20 2
CEPH 1O_20_3O 3
CEPH F 1482
CEPH 2O 12
CEPH F_10 51


In [25]:
print("Finished")

Finished
